In [1]:
import pandas as pd
import requests
from splinter import Browser
from bs4 import BeautifulSoup
import time
from sqlalchemy import create_engine
import json
import pymongo
from pymongo import MongoClient

# Extract CSVs into DataFrames

In [2]:
yelp_file = "../Resources/yelp_data.csv"
yelp_df = pd.read_csv(yelp_file)
yelp_df

,Unnamed: 0,restaurant_name,restaurant_url,restaurant_rating,price_cuisine,address1,address2
0,0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869"
1,1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869"
2,2,Missy’s Main Street Cafe,https://www.yelp.com/biz/missys-main-street-ca...,5.0,"$Breakfast & Brunch, Cafes",181 E Main St,"Randolph, NJ 07869"
3,3,The Corner Bistro,https://www.yelp.com/biz/the-corner-bistro-ran...,4.0,"$$Diners, Italian",477 NJ-10,"Randolph, NJ 07869"
4,4,Beanbury,https://www.yelp.com/biz/beanbury-succasunna?o...,5.0,"Coffee & Tea, Breakfast & Brunch",37 Route 10 E,"Randolph, NJ 07869"
...,...,...,...,...,...,...,...
235,235,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Randolph, NJ 07869"
236,236,LongHorn Steakhouse,https://www.yelp.com/biz/longhorn-steakhouse-f...,3.5,"$$Steakhouses, American (Traditional), Barbeque",50-J International Dr S,"Randolph, NJ 07869"
237,237,Hunan Chinese Room,https://www.yelp.com/biz/hunan-chinese-room-mo...,3.5,$$Chinese,255 Speedwell Ave,"Randolph, NJ 07869"
238,238,Brenda Lee Restaurant,https://www.yelp.com/biz/brenda-lee-restaurant...,3.5,Mexican,15 E Blackwell St,"Randolph, NJ 07869"


In [3]:
# All columns
yelp_df.columns

Index(['Unnamed: 0', 'restaurant_name', 'restaurant_url', 'restaurant_rating',
       'price_cuisine', 'address1', 'address2'],
      dtype='object')

In [15]:
# restaurant_name check
yelp_df["restaurant_name"].value_counts()
#yelp_df["restaurant_name"].count()

Sandwiches Unlimited Lunch Box            2
L & L Deli & Catering                     1
The 53 Grill                              1
JA Spice Island Jerk and American Food    1
Carver’s                                  1
                                         ..
Malay                                     1
Tierras y Sabores Restaurant              1
Paisano’s Pizzeria                        1
Bryans Luncheonette                       1
Blossom Asian Bistro                      1
Name: restaurant_name, Length: 239, dtype: int64

# Dropping duplicate restaurant_name value

In [21]:
yelp_df1= yelp_df.drop_duplicates(subset=['restaurant_name'])
yelp_df1

,Unnamed: 0,restaurant_name,restaurant_url,restaurant_rating,price_cuisine,address1,address2
0,0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869"
1,1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869"
2,2,Missy’s Main Street Cafe,https://www.yelp.com/biz/missys-main-street-ca...,5.0,"$Breakfast & Brunch, Cafes",181 E Main St,"Randolph, NJ 07869"
3,3,The Corner Bistro,https://www.yelp.com/biz/the-corner-bistro-ran...,4.0,"$$Diners, Italian",477 NJ-10,"Randolph, NJ 07869"
4,4,Beanbury,https://www.yelp.com/biz/beanbury-succasunna?o...,5.0,"Coffee & Tea, Breakfast & Brunch",37 Route 10 E,"Randolph, NJ 07869"
...,...,...,...,...,...,...,...
235,235,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Randolph, NJ 07869"
236,236,LongHorn Steakhouse,https://www.yelp.com/biz/longhorn-steakhouse-f...,3.5,"$$Steakhouses, American (Traditional), Barbeque",50-J International Dr S,"Randolph, NJ 07869"
237,237,Hunan Chinese Room,https://www.yelp.com/biz/hunan-chinese-room-mo...,3.5,$$Chinese,255 Speedwell Ave,"Randolph, NJ 07869"
238,238,Brenda Lee Restaurant,https://www.yelp.com/biz/brenda-lee-restaurant...,3.5,Mexican,15 E Blackwell St,"Randolph, NJ 07869"


# Dropping duplicate address1 value

In [24]:
# address1 check
yelp_df["address1"].value_counts()

330 S Salem St          3
1250 Sussex Turnpike    2
39 W Clinton St         2
459 Main St             2
6B S Warren St          1
                       ..
25 E Main St            1
242 E Union Tpke        1
42 Main St              1
517 E Rte 10            1
12 Elm St               1
Name: address1, Length: 235, dtype: int64

In [23]:
yelp_df2= yelp_df.drop_duplicates(subset=['address1'])
yelp_df2

,Unnamed: 0,restaurant_name,restaurant_url,restaurant_rating,price_cuisine,address1,address2
0,0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869"
1,1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869"
2,2,Missy’s Main Street Cafe,https://www.yelp.com/biz/missys-main-street-ca...,5.0,"$Breakfast & Brunch, Cafes",181 E Main St,"Randolph, NJ 07869"
3,3,The Corner Bistro,https://www.yelp.com/biz/the-corner-bistro-ran...,4.0,"$$Diners, Italian",477 NJ-10,"Randolph, NJ 07869"
4,4,Beanbury,https://www.yelp.com/biz/beanbury-succasunna?o...,5.0,"Coffee & Tea, Breakfast & Brunch",37 Route 10 E,"Randolph, NJ 07869"
...,...,...,...,...,...,...,...
235,235,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Randolph, NJ 07869"
236,236,LongHorn Steakhouse,https://www.yelp.com/biz/longhorn-steakhouse-f...,3.5,"$$Steakhouses, American (Traditional), Barbeque",50-J International Dr S,"Randolph, NJ 07869"
237,237,Hunan Chinese Room,https://www.yelp.com/biz/hunan-chinese-room-mo...,3.5,$$Chinese,255 Speedwell Ave,"Randolph, NJ 07869"
238,238,Brenda Lee Restaurant,https://www.yelp.com/biz/brenda-lee-restaurant...,3.5,Mexican,15 E Blackwell St,"Randolph, NJ 07869"


# Dropping duplicate restaurant_name and also address1 value

In [25]:
yelp_df3= yelp_df1.drop_duplicates(subset=['address1'])
yelp_df3

,Unnamed: 0,restaurant_name,restaurant_url,restaurant_rating,price_cuisine,address1,address2
0,0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869"
1,1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869"
2,2,Missy’s Main Street Cafe,https://www.yelp.com/biz/missys-main-street-ca...,5.0,"$Breakfast & Brunch, Cafes",181 E Main St,"Randolph, NJ 07869"
3,3,The Corner Bistro,https://www.yelp.com/biz/the-corner-bistro-ran...,4.0,"$$Diners, Italian",477 NJ-10,"Randolph, NJ 07869"
4,4,Beanbury,https://www.yelp.com/biz/beanbury-succasunna?o...,5.0,"Coffee & Tea, Breakfast & Brunch",37 Route 10 E,"Randolph, NJ 07869"
...,...,...,...,...,...,...,...
235,235,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Randolph, NJ 07869"
236,236,LongHorn Steakhouse,https://www.yelp.com/biz/longhorn-steakhouse-f...,3.5,"$$Steakhouses, American (Traditional), Barbeque",50-J International Dr S,"Randolph, NJ 07869"
237,237,Hunan Chinese Room,https://www.yelp.com/biz/hunan-chinese-room-mo...,3.5,$$Chinese,255 Speedwell Ave,"Randolph, NJ 07869"
238,238,Brenda Lee Restaurant,https://www.yelp.com/biz/brenda-lee-restaurant...,3.5,Mexican,15 E Blackwell St,"Randolph, NJ 07869"


In [33]:
# average rating
average_rating = yelp_df3["restaurant_rating"].sum()/yelp_df3["restaurant_name"].count()
print("average_rating")
print(average_rating)


average_rating
4.0405982905982905


# Create new data with select columns

In [34]:
new_yelp_df = yelp_df3[['restaurant_name', 'address1', 'restaurant_rating']].copy()
new_yelp_df

,restaurant_name,address1,restaurant_rating
0,Rosie’s Trattoria,1181 Sussex Tpke,4.5
1,SubUrban Bar & Kitchen,500 NJ-10,4.0
2,Missy’s Main Street Cafe,181 E Main St,5.0
3,The Corner Bistro,477 NJ-10,4.0
4,Beanbury,37 Route 10 E,5.0
...,...,...,...
235,Clean Juice,68 South St,4.5
236,LongHorn Steakhouse,50-J International Dr S,3.5
237,Hunan Chinese Room,255 Speedwell Ave,3.5
238,Brenda Lee Restaurant,15 E Blackwell St,3.5


# Clean DataFrame

In [36]:
# There are different length for restaurant_name and address1 values.So,we need to fill the empty values.
new_yelp_df['restaurant_name'] = new_yelp_df['restaurant_name'].fillna(0)
new_yelp_df['address1'] = new_yelp_df['address1'].fillna(0)
new_yelp_df['restaurant_rating'] = new_yelp_df['restaurant_rating'].fillna(0)
new_yelp_df

,restaurant_name,address1,restaurant_rating
0,Rosie’s Trattoria,1181 Sussex Tpke,4.5
1,SubUrban Bar & Kitchen,500 NJ-10,4.0
2,Missy’s Main Street Cafe,181 E Main St,5.0
3,The Corner Bistro,477 NJ-10,4.0
4,Beanbury,37 Route 10 E,5.0
...,...,...,...
235,Clean Juice,68 South St,4.5
236,LongHorn Steakhouse,50-J International Dr S,3.5
237,Hunan Chinese Room,255 Speedwell Ave,3.5
238,Brenda Lee Restaurant,15 E Blackwell St,3.5


# Store CSV data into a DataFrame

In [39]:
csv_file = "../Resources/google_data.csv"
google_data_df = pd.read_csv(csv_file)
google_data_df

,Unnamed: 0,restaurant_name,restaurant_url,place_id,price_level,restaurant_rating,address
0,0,Popeyes Louisiana Kitchen,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ3dh_VfcKw4kR3lrLsxKB7tY,1.0,3.7,Rockaway Townsquare Mall-301 Mt Hope Road - Su...
1,1,Buffalo Wild Wings,"<a href=""https://maps.google.com/maps/contrib/...",ChIJAYlPufkKw4kRr7-IhJCnC3Y,2.0,4.2,"343 Mount Hope Avenue, Rockaway"
2,2,Auntie Anne's,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ3fcDtfsKw4kRsy2xOBfI0aQ,1.0,4.2,"301 Mount Hope Avenue Suite 1048, Rockaway"
3,3,Tabor Road Tavern,"<a href=""https://maps.google.com/maps/contrib/...",ChIJg8StYWinw4kRMFNslu1L9xg,3.0,4.4,"510 Tabor Road, Morris Plains"
4,4,Subway,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ6YlEVO8Jw4kR4n_aDP4NbIc,1.0,3.4,"559 East Main Street Store # 2, Denville"
...,...,...,...,...,...,...,...
355,355,Anthony & Sons Bakery & Italian Deli,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ1yZeabefw4kRfvmIeEYa2xY,2.0,4.5,"15 New Jersey 10, Succasunna"
356,356,Pizza Express,"<a href=""https://maps.google.com/maps/contrib/...",ChIJM4tBOgJ1w4kRa9U1mz6xgfI,1.0,3.7,"312 Howard Boulevard, Mount Arlington"
357,357,Shell,"<a href=""https://maps.google.com/maps/contrib/...",ChIJWf-qdf2fw4kR_MdkDkTL6Jo,2.0,3.1,"274 U.S. 46, Mine Hill Township"
358,358,Dunkin',"<a href=""https://maps.google.com/maps/contrib/...",ChIJNcJSc_2fw4kRIA4rycWRc58,1.0,4.2,"274 Rte 46, Mine Hill Township"


In [40]:
# All columns
google_data_df.columns

Index(['Unnamed: 0', 'restaurant_name', 'restaurant_url', 'place_id',
       'price_level', 'restaurant_rating', 'address'],
      dtype='object')

In [42]:
# restaurant name check
google_data_df["restaurant_name"].value_counts()

Subway                                12
Dunkin'                               11
Starbucks                             10
Panera Bread                           5
Domino's Pizza                         4
                                      ..
Tito's Burritos & Wings Morristown     1
Five Guys                              1
Aikou restaurant                       1
Riviera Maya Mexican Restaurant        1
Casella's Pizzeria & Deli              1
Name: restaurant_name, Length: 245, dtype: int64

In [43]:
# restaurant address check
google_data_df["address"].value_counts()

3 Speedwell Avenue, Morristown         6
40 West Park Place, Morristown         4
641 Shunpike Road, Chatham Township    4
1 Convent Road, Morristown             4
184 Columbia Turnpike, Florham Park    3
                                      ..
92 Parsippany Boulevard, Boonton       1
58 West, U.S. 46, Pine Brook           1
160 East Main Street, Rockaway         1
270 U.S. 206, Chester                  1
57 New Jersey 10 E, Succasunna         1
Name: address, Length: 274, dtype: int64

# Create new data with select columns

In [41]:
new_yelp_business_df = yelp_business_df[['name', 'address']].copy()
new_yelp_business_df

,name,address
0,The Range At Lake Norman,10913 Bailey Rd
1,"Carlos Santo, NMD","8880 E Via Linda, Ste 107"
2,Felinus,3554 Rue Notre-Dame O
3,Nevada House of Hose,1015 Sharp Cir
4,USE MY GUY SERVICES LLC,4827 E Downing Cir
...,...,...
209388,Nishi Sushi,9750 Weston Road
209389,Walmart,3240 Wilkinson Blvd
209390,Five Guys,7014-590 E Camelback Rd
209391,Indian Trail Dog Training,


# Clean DataFrame

In [42]:
new_yelp_business_df['name'] = new_yelp_business_df['name'].fillna(0)
new_yelp_business_df['address'] = new_yelp_business_df['address'].fillna(0)
new_yelp_business_df

,name,address
0,The Range At Lake Norman,10913 Bailey Rd
1,"Carlos Santo, NMD","8880 E Via Linda, Ste 107"
2,Felinus,3554 Rue Notre-Dame O
3,Nevada House of Hose,1015 Sharp Cir
4,USE MY GUY SERVICES LLC,4827 E Downing Cir
...,...,...
209388,Nishi Sushi,9750 Weston Road
209389,Walmart,3240 Wilkinson Blvd
209390,Five Guys,7014-590 E Camelback Rd
209391,Indian Trail Dog Training,


In [37]:
yelp_business_df1 = yelp_business_df.loc[yelp_business_df["name"] == "new_yelp_df['restaurant_name']"]
yelp_business_df1.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours


# Create database connection

In [63]:
# Making a Connection with MongoClient
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
# database
mydb = myclient["NJ_Restaurants"]

In [64]:
# Check if Database Exists
print(myclient.list_database_names())

['ClassDB', 'Fruits_db', 'NJ_Restaurants', 'admin', 'config', 'craigslist_db', 'local', 'mars_db', 'store_inventory', 'team_db']


# Creating a Collection

In [65]:
mycol = mydb["Restaurants"]

In [66]:
# Check if Collection Exists
print(mydb.list_collection_names())

['Restaurants']


# Load DataFrames into database

In [67]:
# Load csv dataset
data = pd.read_csv('../Resources/yelp_data.csv')
data.reset_index(inplace=True)
data_dict = data.to_dict("records")

# Insert collection
mycol.insert_many(data_dict)